In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import warnings
from review import review_parsing
from db import db_insert
import random
warnings.filterwarnings('ignore')

In [2]:
# 영화 dataframe
# 페이지 접속
driver = webdriver.Chrome()
driver.get("https://m.kinolights.com/discover/explore") # 키노라이트 탐색를 여는 녀석
time.sleep(0.3) #1초 웨이팅 없으면 리젝당할수도...

# 메뉴얼 필터링

In [4]:
import json


# 영화정보 크롤링 function
def crawling():
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    links = driver.find_elements_by_xpath('//*[@id="contents"]/section[1]/div/div/ul/li[1]/div/a')
    link_len = len(driver.find_elements_by_xpath(f'//*[@id="contents"]/section[2]/div/div/ul/li'))
    ottLinks = []
    if link_len == 0:
        link_len = len(driver.find_elements_by_xpath(f'//*[@id="contents"]/section[1]/div/div/ul/li'))
        for i in range(link_len):
            link = driver.find_elements_by_xpath(f'//*[@id="contents"]/section[1]/div/div/ul/li[{i+1}]/div/a')[0]
            ottLinks.append({"Ott":f'{link.text.split(" ")[0]}',"OttLink":link.get_attribute('href')})
            # LINKS += "{name:"+f'"{link.text.split(" ")[0]}"'+":'"+link.get_attribute('href')+"'},"
    else:
        link_len = len(driver.find_elements_by_xpath(f'//*[@id="contents"]/section[2]/div/div/ul/li'))
        for i in range(link_len):
            link = driver.find_elements_by_xpath(f'//*[@id="contents"]/section[2]/div/div/ul/li[{i+1}]/div/a')[0]
            ottLinks.append({"Name":f'{link.text.split(" ")[0]}',"OttLink":link.get_attribute('href')})
    casts = []
    try:
        names = soup.select('.photo > img ')
        photos = soup.select('.photo > img ')
        # director = '"'+names[0]['alt']+'"'+':"'+photos[0]['src']+'"'
        director = {"Director":f'{str(names[0]["alt"])}', "ProfilImg":f'{photos[0]["src"]}' }
        # LK:: Link { name:value, photo:value } ,
        for i in range(len(photos)-1):
            if i < len(photos)-2:
                casts.append({"Actor":f'{names[i+1]["alt"]}',"ProfilImg":f'{photos[i+1]["src"]}'})
                # CASTS += '{"'+names[i+1]['alt']+'":"'+photos[i+1]["src"]+'"},'
            else:
                casts.append({"Actor":f'{names[i+1]["alt"]}',"ProfilImg":f'{photos[i+1]["src"]}'})
    except IndexError:
        casts = None
        director = None
    try:
        trailer = driver.find_element_by_tag_name('iframe').get_attribute('src')
    except:
        trailer = None

    dict = { 'TitleKr' : driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[1]/h3').text,
            'TitleEn' : driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/h4")[0].text,
            'Poster' : driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[2]/img').get_attribute('src'),
            'Genre' : driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/p/span[1]")[0].text[:-2],
            'Year' : driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/p/span[2]")[0].text,
            'Kino' : None if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[1]/div")[0].text[:-1] =='- ' else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[1]/div")[0].text[:-1]),
            'Tomato' : None if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[1]/div")[0].text[:-3]=='- '
            else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[1]/div")[0].text[:-3]),
            'Imdb' :None if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[2]/div")[0].text[:-2]=='-'
             else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[2]/div")[0].text[:-2]),
            'Star' : None  if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/div/div")[0].text[:]=='-'
            else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/div/div")[0].text[:]),
            'OttLinks' : ottLinks,
            'Director' :  json.dumps(director),
            'Casts' : casts,
            'Trailer' : json.dumps(trailer),
            'KinoId': int(driver.current_url.split("/")[-1]) }
    return dict

# 스크롤다운

In [5]:
time.sleep(3)
#  스크롤다운
SCROLL_PAUSE_SEC = 1
# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # try:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("스크롤 완료!")
        break
    last_height = new_height
    # except:
    #     print("스크롤 완료")
    #     break
#iframe ? 스크롤완료시 오류 없음

스크롤 완료!


# get link

In [6]:
from selenium.webdriver import ActionChains
cnt = 0
links = []
# 맄크 가져오기
while True:
    try:
        cnt += 1
        url = f'//*[@id="listArea"]/div[3]/div[2]/div[{cnt}]/a'
        url = f'//*[@id="contents"]/main/div[1]/div[3]/div[2]/div[{cnt}]/a'
        link = driver.find_element_by_xpath(url).get_attribute('href')
        links.append(link)
        # driver.get(link)
        # time.sleep(0.3)
        # print(crawling())
        # driver.back()
        # time.sleep(0.3)
        # if cnt==3:
        #     break
    except:
        break
# for link in links:
#     #print(link)
print(f'{len(links)} 가져옴!')

8 가져옴!


# main

## get req

# main 함수

In [7]:
def main(inputlinks):
    i = 0
    s = 0
    e = 0
    err_links =[]
    while True:
        try:
            time.sleep(random.randrange(1,7))
            driver.get(inputlinks[i])
            time.sleep(random.randrange(1,3))
            m_df =  pd.DataFrame(columns = [ 'TitleKr', 'TitleEn', 'Poster','Genre', 'Year', 'Kino', 'Tomato', 'Imdb', 'Star','OttLinks', 'Info', 'Director', 'Casts', 'Trailer', 'KinoId'])
            m_df = m_df.append(crawling(), ignore_index=True)
            m_df['OttLinks'] = m_df['OttLinks'].astype('str')
            m_df['Casts'] = m_df['Casts'].astype('str')
            db_insert(m_df,'MovieInfo')
            kino_id = inputlinks[i].split("/")[-1]
            try:
                db_insert(review_parsing(int(kino_id)),'MovieReview')
            except:
                print(f'get review err {kino_id}')
            i += 1
            s += 1
            print(f'{i}/{len(inputlinks)} 완료!')
        except IndexError as id:
            print(f'크롤링 완료~~~~~~~~~!!!!!!!!!!!!!!!{s}/{e}')
            break
        except all as msg:
            print(f'{i}번 err: {msg}')
            i += 1
            e += 1
            err_links.append(links[i])
            continue
        # if i==2:
        #     break
    return err_links

# RUN

In [8]:
err_links = main(links)
print(f'{len(err_links)} 개 에러 링크 취득!')
err_links

208 insert success
156 insert success
1/8 완료!
209 insert success
get review err 100200
2/8 완료!
210 insert success
10 insert success
3/8 완료!
211 insert success
17 insert success
4/8 완료!
212 insert success
24 insert success
5/8 완료!
213 insert success
34 insert success
6/8 완료!
214 insert success
36 insert success
7/8 완료!
215 insert success
38 insert success
8/8 완료!
크롤링 완료~~~~~~~~~!!!!!!!!!!!!!!!8/0
0 개 에러 링크 취득!


[]

In [ ]:
def err_check(err_link):
    while True:
            time.sleep(random.randrange(1,7))
            driver.get(err_link)
            time.sleep(random.randrange(1,3))
            m_df =  pd.DataFrame(columns = [ 'TITLE_KR', 'TITLE_EN', 'POSTER','GENRE', 'YEAR', 'KINO', 'TOMATO', 'IMDB', 'STAR','LINK', 'INFO', 'DIRECTOR', 'CAST', 'TRAILER', 'KINO_ID'])
            m_df = m_df.append(crawling(), ignore_index=True)
            kino_id = err_link.split("/")[-1]
            db_insert(m_df,'movie_info')
            db_insert(review_parsing(int(kino_id)),'movie_review')
            print(f'{i}번 완료!')

In [ ]:
err_check(err_links[0])